In [ ]:
#This notebook reads the GloVe data and shows that King:man =  Queen : Woman
#, but some analogies do not wark as also shown.
# This reads from glove 50dimesnion file, and finds nearest word to a analogy
#Though it finds the missing word correctly for most words, for some it does not
#R Chandrashekar

In [18]:
import numpy as np


In [19]:
def read_glove_data(glove_file):
    with open(glove_file, 'r', encoding="utf-8") as f:
        words = set()
        word_to_vec_map = {}
        curr_word=None
        i = 0
        try:
            for line in f:
                i+=1
                try:
                    line = line.strip().split()
                    curr_word = line[0]
                    word_to_vec_map[curr_word] = np.array(line[1:], dtype=np.float64)
                except Exception as E:
                    print ("got An exception, word=", curr_word, i)
                    pass         
        except Exception as E:
            print ("got An exception before for, word=", curr_word, i)
            pass         
            
    return word_to_vec_map

In [20]:
word_to_vec_map = read_glove_data('glove.6B.50d.txt')

In [21]:
len(word_to_vec_map)

224761

In [22]:
def cos_similarity(vec_u, vec_v):
    distance = 0.0   
    dot = np.inner(vec_u,vec_v)
    norm_vec_u = np.linalg.norm(vec_u)
    norm_vec_v = np.linalg.norm(vec_v)
    cos_similarity = dot/(norm_vec_u*norm_vec_v)
    
    return cos_similarity

In [23]:
fatherVec = word_to_vec_map["father"]
motherVec = word_to_vec_map["mother"]
ballVec = word_to_vec_map["ball"]
crocodileVec = word_to_vec_map["crocodile"]
franceVec = word_to_vec_map["france"]
italyVec = word_to_vec_map["italy"]
parisVec = word_to_vec_map["paris"]
romeVec = word_to_vec_map["rome"]

print("cos_similarity(father, mother) = ", cos_similarity(fatherVec, motherVec))
print("cos_similarity(ball, crocodile) = ",cos_similarity(ballVec, crocodileVec))
print("cos_similarity(france - paris, rome - italy) = ",cos_similarity(franceVec - parisVec, 
                                                                          italyVec - romeVec))
print("cos_similarity(france - paris, rome - italy) = ",cos_similarity(franceVec - parisVec, 
                                                                          romeVec - italyVec))

cos_similarity(father, mother) =  0.8909038442893615
cos_similarity(ball, crocodile) =  0.2743924626137942
cos_similarity(france - paris, rome - italy) =  0.6751479308174202
cos_similarity(france - paris, rome - italy) =  -0.6751479308174202


In [24]:
def findMissing(word_a, word_b, word_c, word_to_vec_map):
    word_a, word_b, word_c = word_a.lower(), word_b.lower(), word_c.lower()
    
    e_a, e_b, e_c = word_to_vec_map[word_a], word_to_vec_map[word_b], word_to_vec_map[word_c]#None
    words = word_to_vec_map.keys()
    max_cosine_sim = -1000              
    best_word = None                   

    for w in words:        
        if w in [word_a, word_b, word_c] :
            continue
        
        try:
            cosine_sim = cos_similarity(e_b-e_a, word_to_vec_map[w]-e_c)
        
            if (cosine_sim > max_cosine_sim):
                max_cosine_sim = cosine_sim
                best_word = w
        except ValueError as ve:
            print ("Got an exception", ve, w)
            pass
        except KeyError as ke:
            print ("this key", w, "not found")
            
    print ("Done")    
    return best_word

word_to_vec_map["usa"]

In [25]:
word_a, word_b, word_c= 'india', 'delhi', 'france'
w = "paris"
e_a, e_b, e_c = word_to_vec_map[word_a], word_to_vec_map[word_b], word_to_vec_map[word_c]
cosine_sim = cos_similarity(e_b-e_a, word_to_vec_map[w]-e_c)
print (cosine_sim ," for paris")
w = "nanterre"
cosine_sim = cos_similarity(e_b-e_a, word_to_vec_map[w]-e_c)
print (cosine_sim ," for nanterre", type(e_a), e_a.shape)


0.6958505344885514  for paris
0.7249669057754777  for nanterre <class 'numpy.ndarray'> (50,)


In [26]:
puzzle_triads= [('italy', 'italian', 'spain'), 
                 ('father', 'mother', 'son'),
                 ('brother', 'sister', 'nephew'),
                 ('india', 'delhi', 'japan'), 
                 ('man', 'woman', 'boy'), 
                 ('small', 'smaller', 'large'),
                ('king', 'man', 'queen'),
                 ('king', 'woman', 'queen')
                ]
for t in puzzle_triads:
    print (t, findMissing(t[0], t[1], t[2], word_to_vec_map))

Done
('italy', 'italian', 'spain') spanish
Done
('father', 'mother', 'son') daughter
Done
('brother', 'sister', 'nephew') niece
Done
('india', 'delhi', 'japan') tokyo
Done
('man', 'woman', 'boy') girl
Done
('small', 'smaller', 'large') larger
Done
('king', 'man', 'queen') woman
Done
('king', 'woman', 'queen') girl
